In [34]:
import time
import mlflow
import pandas as pd
from sklearn.model_selection import train_test_split
from mlflow.models.signature import infer_signature
from xgboost import XGBRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import (
    accuracy_score,
    f1_score,
    ConfusionMatrixDisplay,
    RocCurveDisplay,
    classification_report
)

In [35]:
dataset = pd.read_csv('get_around_pricing_project.csv').drop('Unnamed: 0', axis=1)
dataset

FileNotFoundError: [Errno 2] No such file or directory: 'template_dataset.csv'

In [16]:
dataset.loc[2].to_list()

['Citroën',
 183297,
 120,
 'diesel',
 'white',
 'convertible',
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 101]

In [33]:
# mlflow server connection
mlflow.set_tracking_uri("http://0.0.0.0:443/")
#mlflow.set_tracking_uri("../mlruns")
EXPERIMENT_NAME="getaround_predict_local"
mlflow.set_experiment(EXPERIMENT_NAME)
experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
if experiment:
    print("Experiment ID:", experiment.experiment_id)
    print("Artifact Location:", experiment.artifact_location)
else:
    print(f"Experiment '{EXPERIMENT_NAME}' does not exist.")

# start experiment time tracking
start_time = time.time()
mlflow.sklearn.autolog(log_models=False)

# load dataset for training
dataset = pd.read_csv('get_around_pricing_project.csv').drop('Unnamed: 0', axis=1)
Y = dataset['rental_price_per_day']
X = dataset.drop('rental_price_per_day', axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2, random_state=1)

# list numeric/categorical columns
numeric_features = []
categorical_features = []
for i, t in X.dtypes.items():
    if ("float" in str(t)) or ("int" in str(t)):
        numeric_features.append(i)
    else:
        categorical_features.append(i)
# list check
print("Found numeric features ", numeric_features)
print("Found categorical features ", categorical_features)

# assemble the model with preprocessing
preprocessor = ColumnTransformer(
    transformers=[
        ("num", StandardScaler(), numeric_features),
        ("cat", OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_features),
    ]
)
model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("regressor", XGBRegressor())
])

# train the model
with mlflow.start_run(experiment_id = experiment.experiment_id):
    model.fit(X_train, Y_train)
    predictions = model.predict(X_train)

    mlflow.sklearn.log_model(model, "xgboost")
    '''mlflow.sklearn.log_model(
        sk_model=model,
        artifact_path="getaround_price_prediction",
        registered_model_name="xgboost",
        signature=infer_signature(X_train, predictions)
    )'''

print("...Done!")
print(f"---Total training time: {time.time()-start_time}")
print('train_score', model.score(X_train, Y_train))
print('test_score', model.score(X_test, Y_test))

mlflow.end_run()

MlflowException: API request to endpoint /api/2.0/mlflow/experiments/get-by-name failed with error code 401 != 200. Response body: 'You are not authenticated. Please see https://www.mlflow.org/docs/latest/auth/index.html#authenticating-to-mlflow on how to authenticate.'

In [11]:
dataset.dtypes.to_list()


[dtype('O'),
 dtype('int64'),
 dtype('int64'),
 dtype('O'),
 dtype('O'),
 dtype('O'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('bool'),
 dtype('int64')]

In [32]:
mlflow.set_tracking_uri("https://mlflow-s3-5c46c0d9d46b.herokuapp.com/")
# Read data 
input = dataset.loc[2].to_list()    
print(input)

columns = ['model_key', 'mileage', 'engine_power', 'fuel', 'paint_color',
       'car_type', 'private_parking_available', 'has_gps',
       'has_air_conditioning', 'automatic_car', 'has_getaround_connect',
       'has_speed_regulator', 'winter_tires', 'rental_price_per_day']
features = pd.DataFrame([input], 
                        columns=columns,
                        )
display('features',features)

logged_model = 'runs:/18f2bcb978c2417cb3c6f85174da829e/xgboost'
#logged_model = 'logged_model/getaround_price_prediction'

# Load model as a PyFuncModel.
loaded_model = mlflow.pyfunc.load_model(logged_model)

prediction = loaded_model.predict(features)
print('prediction',prediction)

# Format response
response = {"prediction": prediction.tolist()[0]}
response

['Citroën', 183297, 120, 'diesel', 'white', 'convertible', False, False, False, False, True, False, True, 101]


'features'

,model_key,mileage,engine_power,fuel,paint_color,car_type,private_parking_available,has_gps,has_air_conditioning,automatic_car,has_getaround_connect,has_speed_regulator,winter_tires,rental_price_per_day
0,Citroën,183297,120,diesel,white,convertible,False,False,False,False,True,False,True,101


2024/06/03 16:38:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/arthurblanchon/Library/Python/3.9/lib/python/site-packages/mlflow/types/utils.py:394: UserWarning: Hint: Inferred schema contains integer column(s). Integer columns in Python cannot represent missing values. If your input data contains missing values at inference time, it will be encoded as floats and will cause a schema enforcement error. The best way to avoid this problem is to infer the model schema based on a realistic data sample (training dataset) that includes missing values. Alternatively, you can declare integer columns as doubles (float64) whenever these columns may have missing values. See `Handling Integers With Missing Values <https://www.mlflow.org/docs/latest/models.html#handling-integers-with-missing-values>`_ for more details."


prediction [110.788]


{'prediction': 110.78800201416016}

In [32]:
import os
from mlflow.tracking import MlflowClient
mlflow.set_tracking_uri("https://mlflow-server-09febdc5a39a.herokuapp.com")

# Initialize MLflow client
client = MlflowClient()

# Specify the local directory where you want to store artifacts
local_dir = "logged_model"

# Create the local directory if it doesn't exist
if not os.path.exists(local_dir):
    os.mkdir(local_dir)

# Assume you have logged an artifact named "features.txt" during an MLflow run
getaround_price_prediction = '''model_key, mileage, engine_power, fuel, paint_color,
       car_type, private_parking_available, has_gps,
       has_air_conditioning, automatic_car, has_getaround_connect,
       has_speed_regulator, winter_tires, rental_price_per_day'''
with open("getaround_price_prediction.txt", 'w') as f:
    f.write(getaround_price_prediction)

# Create a sample MLflow run and log the artifact "features.txt"
with mlflow.start_run() as run:
    mlflow.log_artifact("getaround_price_prediction.txt", artifact_path="getaround_price_prediction")

# Download the artifact to local storage
local_path = client.download_artifacts('c956f13b3d334bfb9202f52244a224e9', "getaround_price_prediction", local_dir)
print(f"Artifacts downloaded in: {local_dir}")

Artifacts downloaded in: logged_model
